In [1]:
import numpy as np

import lal
from lal import ComputeDetAMResponse
from lal import CachedDetectors

from lal import LALDetectorIndexLHODIFF,LALDetectorIndexLLODIFF,LALDetectorIndexVIRGODIFF

DETECTOR_SITES = {
    'H1': LALDetectorIndexLHODIFF,
    'L1': LALDetectorIndexLLODIFF,
    'V1': LALDetectorIndexVIRGODIFF
    }

import lalsimulation

import matplotlib.pyplot as plt

In [2]:
class Detector(object):
    """
    A Detector object characterises a gravitational wave (GW) interferometric detector
    """

    def __init__(self, detector):
        """
        detector  -- label string of the detector
        descriptor -- LAL descriptor
        location -- geographic location of the detector
        response -- response matrix

        """
        self.name = detector
        self.descriptor =  CachedDetectors[DETECTOR_SITES[detector]]
        self.location = lalsimulation.DetectorPrefixToLALDetector(detector).location
        self.response = lalsimulation.DetectorPrefixToLALDetector(detector).response
        
    def antenna_pattern(self, time_at_coalescence, RA, dec):
        
        gmst_rad = lal.GreenwichMeanSiderealTime(time_at_coalescence)
        """fplus = []
        fcross = []
        for (RA_val, dec_val) in zip(RA, dec):
            fplus_val,fcross_val = ComputeDetAMResponse(self.descriptor.response,
                                    RA_val, dec_val, psi, gmst_rad)
            fplus.append(fplus_val)
            fcross.append(fcross_val)

        return np.array(fplus), np.array(fcross)
        """
        fplus,fcross = ComputeDetAMResponse(self.descriptor.response,
                                    RA, dec, psi, gmst_rad)
        
        return fplus,fcross
        
        
    def time_delay_from_earth_center(self, RA, dec, time_gps):
        """ Returns the time delay from the earth center
        """
        return lal.TimeDelayFromEarthCenter(self.location,
                      float(RA), float(dec), float(time_gps))

In [77]:
#constants
msun = 4.926e-6 #s-1       #1.988*10**30 #kg
c = 1                     #3*10**8 #m/s
G = 1                     #6.674*10**(-11)#N/kg**2*m**2
parsec = 3.086*10**16/(2.998*10**8) #s

#parameters
m1 = 1.46*msun
m2 =  1.26*msun
iota = 170.*np.pi/180
v = np.cos(iota)
psi = 0.*np.pi/180
D = 40*10**6*parsec
tc = 1187008882.43
phic = 0.*np.pi/180
time_at_coalescence = tc
mchirp = (m1*m2)**(3./5.)/(m1+m2)**(1./5.)

ra = 168*np.pi/180
dec = 61*np.pi/180



"""
#injection 4532 only H1
m1 = 1.43*msun
m2 = 1.57*msun
iota = 99.*np.pi/180
v = np.cos(iota)
psi = 152.*np.pi/180
D = 127*10**6*parsec
tc = 966864415
phic = 25.*np.pi/180
time_at_coalescence = tc
mchirp = (m1*m2)**(3./5.)/(m1+m2)**(1./5.)

ra = 140.8*np.pi/180
dec = 70.*np.pi/180

"""

"""
#injection 42544 H1, L1 2010-12-12 4h41min16.6s
m1 = 1.36*msun
m2 = 1.45*msun
iota = 113.*np.pi/180
v = np.cos(iota)
psi = 212.*np.pi/180
D = 22*10**6*parsec
tc = 970893690
phic = 120.*np.pi/180
time_at_coalescence = tc
mchirp = (m1*m2)**(3./5.)/(m1+m2)**(1./5.)

ra = 10.7*np.pi/180
dec = 113.*np.pi/180
"""

'\n#injection 42544 H1, L1 2010-12-12 4h41min16.6s\nm1 = 1.36*msun\nm2 = 1.45*msun\niota = 113.*np.pi/180\nv = np.cos(iota)\npsi = 212.*np.pi/180\nD = 22*10**6*parsec\ntc = 970893690\nphic = 120.*np.pi/180\ntime_at_coalescence = tc\nmchirp = (m1*m2)**(3./5.)/(m1+m2)**(1./5.)\n\nra = 10.7*np.pi/180\ndec = 113.*np.pi/180\n'

In [59]:
#some useful functions

def antenna(Fplus,Fcross,v):

    output = (1+v**2)/2*Fplus-1j*v*Fcross
    
    return output


def strain(f, tau):
    
    Fplus, Fcross = detector.antenna_pattern(time_at_coalescence, ra, dec)
    
    phase = 2.*np.pi*f*tc - phic - np.pi/4. + 3./4.*(8.*np.pi*mchirp*f)**(-5./3.)
    
    h = np.e**(1j*2*np.pi*f*tau)*strain0(f)*antenna(Fplus, Fcross, v)

    return h


def strain0(f):
    
    Fplus, Fcross = detector.antenna_pattern(time_at_coalescence, ra, dec)
    
    phase = 2.*np.pi*f*tc -    phic - np.pi/4. + 3./4.*(8.*np.pi*mchirp*f)**(-5./3.)

    h0 = np.sqrt(5./24.)*np.pi**(-2./3.)*c**(-3/2)*D**(-1)*(G*mchirp)**(5./6.)*f**(-7./6.)*np.e**(1j*phase)
    
    return h0


def derivatives(detector,ra,dec,epsilon):
    #numerical derivatives of Fplus and Fcross
    
    derFplusplus = (((detector.antenna_pattern(time_at_coalescence,ra+epsilon, dec))[0]))
    derFplusminus = (((detector.antenna_pattern(time_at_coalescence,ra-epsilon, dec))[0]))
    derFplus = (derFplusplus-derFplusminus)/(2*epsilon)
    derFplus_ra = float(derFplus)


    derFcrossplus = (((detector.antenna_pattern(time_at_coalescence,ra+epsilon, dec))[1]))
    derFcrossminus = (((detector.antenna_pattern(time_at_coalescence,ra-epsilon, dec))[1]))
    derFcross = (derFcrossplus-derFcrossminus)/(2*epsilon)
    derFcross_ra = float(derFcross)


    derFplusplus = (((detector.antenna_pattern(time_at_coalescence,ra, dec+epsilon))[0]))
    derFplusminus = (((detector.antenna_pattern(time_at_coalescence,ra, dec-epsilon))[0]))
    derFplus = (derFplusplus-derFplusminus)/(2*epsilon)
    derFplus_dec = float(derFplus)


    derFcrossplus = (((detector.antenna_pattern(time_at_coalescence,ra, dec+epsilon))[1]))
    derFcrossminus = (((detector.antenna_pattern(time_at_coalescence,ra, dec-epsilon))[1]))
    derFcross = (derFcrossplus-derFcrossminus)/(2*epsilon)
    derFcross_dec = float(derFcross)
    
    return derFplus_ra, derFcross_ra,derFplus_dec,derFcross_dec 

def derivativestau(detector,ra,dec,epsilon):
    
    #numerical derivative for tau
    dertauplus = detector.time_delay_from_earth_center(ra+epsilon, dec, time_at_coalescence)
    dertauminus = detector.time_delay_from_earth_center(ra-epsilon, dec, time_at_coalescence)
    dertau_ra = (dertauplus - dertauminus)/(2*epsilon)
    
    dertauplus = detector.time_delay_from_earth_center(ra, dec+epsilon, time_at_coalescence)
    dertauminus = detector.time_delay_from_earth_center(ra, dec-epsilon, time_at_coalescence)
    dertau_dec = (dertauplus - dertauminus)/(2*epsilon)
    
    dertauplus = detector.time_delay_from_earth_center(ra, dec, time_at_coalescence+epsilon*100)
    dertauminus = detector.time_delay_from_earth_center(ra, dec, time_at_coalescence-epsilon*100)
    dertau_tc = (dertauplus - dertauminus)/(2*epsilon)  
    
    return dertau_ra, dertau_dec, dertau_tc

def derivativepsi(detector,ra,dec,epsilon):
    
    #numerical of fcross and fplus with respect to psi 
    fplusplus, fcrossplus = ComputeDetAMResponse(detector.descriptor.response, ra, dec, psi+epsilon, tc)
    fplusminus, fcrossminus = ComputeDetAMResponse(detector.descriptor.response, ra, dec, psi-epsilon, tc)

    dfplus_dpsi = (fplusplus-fplusminus)/(2*epsilon)
    dfcross_dpsi = (fcrossplus-fcrossminus)/(2*epsilon)
    
    return dfplus_dpsi, dfcross_dpsi

In [60]:
detectors = ['H1', 'L1', 'V1']

## Fisher Matrix for 8 parameters
Throughout the script the Fisher matrix is calculated in the high SNR approximation 

In [61]:
#Fisher matrix for 8 unknowns: D, v=cosi, psi, theta, phi, tc, phic, mchirp
fval = np.arange(1000)/1. + 20

df = (fval.max()-fval.min())/len(fval)

n = 8

# espilon for the numerical derivatives of Fplus and Fcross
epsilon = 1e-10

F = np.zeros([n,n])

for det in detectors:

    if det is 'L1':
        print 'Livingston'
        Sval = (0.8e-22*(14./(0.08+fval))**2)**2+0.67e-23**2+((fval/2000.)*3.e-23)**2
        detector = Detector('L1')
    if det is 'H1':
        print 'Hanford'
        Sval = ((1e-22*(24./(0.1+fval))**2)**2+0.7e-23**2+((fval/2000.)*3.4e-23)**2)
        detector = Detector('H1')
    if det is 'V1':
        print 'Virgo'
        Sval = (1e-22*(35./(0.1+fval))**2)**2+0.9e-23**2+((fval/2000.)*15e-23)**2
        detector = Detector('V1')
    
    tau = time_delay = detector.time_delay_from_earth_center(ra, dec, time_at_coalescence)
    
    Fplus, Fcross = detector.antenna_pattern(time_at_coalescence, ra, dec)

    #numerical derivatives
    derFplus_ra, derFcross_ra, derFplus_dec, derFcross_dec  = derivatives(detector,ra,dec,epsilon)
    dertau_ra, dertau_dec, dertau_tc = derivativestau(detector,ra,dec,epsilon)
    derFplus_psi, derFcross_psi = derivativepsi(detector,ra,dec,epsilon)


    for f,S in zip(fval,Sval):
        for i in range(n):
            if(i==0):
                dfdpi = -strain(f,tau)/D
            if(i==1):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(v*Fplus-1j*Fcross)
            if(i==2):
                dfdpi = strain(f,tau)*(5/(6*mchirp)-1j*5/4*(8*np.pi*f)**(-5/3)*mchirp**(-8/3))
            if(i==3):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_ra*(1+v**2)/2-1j*v*derFcross_ra)+strain(f,tau)*2*np.pi*1j*f*dertau_ra
            if(i==4):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_dec*(1+v**2)/2-1j*v*derFcross_dec)+strain(f,tau)*2*np.pi*1j*f*dertau_dec
            if(i==5):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
            if(i==6):
                dfdpi = 2j*np.pi*f*(1+dertau_tc)*strain(f,tau)
            if(i==7):
                dfdpi = -1j*strain(f,tau)
            
            for j in range(n):
                if(j==0):
                    dfdpj = -strain(f,tau)/D
                if(j==1):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(v*Fplus-1j*Fcross)
                if(j==2):
                    dfdpj = strain(f,tau)*(5/(6*mchirp)-1j*5/4*(8*np.pi*f)**(-5/3)*mchirp**(-8/3))
                if(j==3):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_ra*(1+v**2)/2-1j*v*derFcross_ra)+strain(f,tau)*2*np.pi*1j*f*dertau_ra
                if(j==4):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_dec*(1+v**2)/2-1j*v*derFcross_dec)+strain(f,tau)*2*np.pi*1j*f*dertau_dec
                if(j==5):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
                if(j==6):
                    dfdpj = 2j*np.pi*f*(1+dertau_tc)*strain(f,tau)
                if(j==7):
                    dfdpj = -1j*strain(f,tau)

                F[i,j] += 4*S**(-1)*(dfdpi.real*dfdpj.real+dfdpi.imag*dfdpj.imag)*df
                
                #F[i,j] += 2*S**(-1)*(dfdpi*dfdpj.conjugate()+dfdpi.conjugate()*dfdpj)*df

I = (np.mat(F).I)
print I

print 'The relative distance error is {0}'.format(np.sqrt(I[0,0]/D**2))
print 'delta cos(iota) = {0}'.format(np.sqrt(I[1,1]))
print 'The relative mchirp error is {0}'.format(np.sqrt(I[2,2]/mchirp**2))
print 'delta ra error = {0}'.format(np.sqrt(I[3,3]))
print 'delta dec error = {0}'.format(np.sqrt(I[4,4]))
print 'delta psi error = {0}'.format(np.sqrt(I[5,5]))
print "delta t_c = {0}".format(np.sqrt(I[6,6]))
print 'delta phi_c error = {0}'.format(np.sqrt(I[7,7]))

Hanford
Livingston
Virgo
[[  6.74269013e+30   2.76864189e+15  -1.84150802e+02  -8.30912143e+12
    3.96730289e+10   4.27298310e+13  -1.57200320e+10   1.35984314e+14]
 [  2.76864189e+15   1.13884345e+00  -7.89793377e-14  -3.85193726e-03
    8.10179215e-05   2.02997021e-02  -8.34069339e-06   4.91940823e-02]
 [ -1.84150802e+02  -7.89793379e-14   4.93810607e-23  -8.70837218e-16
    1.44044051e-15  -9.36933934e-15  -1.72229271e-16  -2.17021148e-13]
 [ -8.30912141e+12  -3.85193726e-03  -8.70837220e-16   2.41627180e-04
    3.42720643e-05  -7.99333364e-04  -1.32371158e-07   1.43986881e-03]
 [  3.96730206e+10   8.10179181e-05   1.44044051e-15   3.42720644e-05
    5.99353813e-05   1.08621759e-04  -9.58131792e-07  -3.92341474e-04]
 [  4.27298308e+13   2.02997020e-02  -9.36933934e-15  -7.99333364e-04
    1.08621759e-04   5.15863572e-03  -3.26278327e-06  -1.10329933e-02]
 [ -1.57200318e+10  -8.34069332e-06  -1.72229271e-16  -1.32371158e-07
   -9.58131792e-07  -3.26278327e-06   1.79694331e-08   1.08

In [26]:
#tests on ordres de grandeur
print (strain(40,tau))
print (strain0(40))

for i in range(8):
    print F[i,i]

(8.73924234717e-24+2.36360842583e-23j)
(-2.26205634697e-23-2.01406436658e-23j)
1.77958554603e-17
339752865.135
9.49887393655e+27
946652623.868
12668863869.6
1814431343.99
6.48009216254e+13
301694531.88


In [46]:
#Fisher matrix for 8 unknowns: D, v=cosi, Mchirp, ra, dec, psi, tc, phic
fval = np.arange(2020)/1. + 20
 
df = (fval.max()-fval.min())/len(fval)

n = 8

# espilon for the numerical derivatives of Fplus and Fcross
epsilon = 1e-10

n_ra = 50

ra_plot = np.linspace(-np.pi, np.pi, n_ra)
dec_plot= np.linspace(-np.pi/2, np.pi/2, n_ra)

ramesh, decmesh = np.meshgrid(ra_plot,dec_plot)

relDerror = []

for (ra,dec) in zip(ramesh.flatten(),decmesh.flatten()):
    F = np.zeros([n,n])
    for det in detectors:

        if det is 'L1':
            #print 'Livingston'
            Sval = (0.8e-22*(14./(0.08+fval))**2)**2+0.67e-23**2+((fval/2000.)*3.e-23)**2
            detector = Detector('L1')
        if det is 'H1':
            #print 'Hanford'
            Sval = ((1e-22*(24./(0.1+fval))**2)**2+0.7e-23**2+((fval/2000.)*3.4e-23)**2)
            detector = Detector('H1')
        if det is 'V1':
            #print 'Virgo'
            Sval = (1e-22*(35./(0.1+fval))**2)**2+0.9e-23**2+((fval/2000.)*15e-23)**2
            detector = Detector('V1')

        tau = time_delay = detector.time_delay_from_earth_center(ra, dec, time_at_coalescence)

        Fplus, Fcross = detector.antenna_pattern(time_at_coalescence, ra, dec)

        #numerical derivatives
        derFplus_ra, derFcross_ra, derFplus_dec, derFcross_dec  = derivatives(detector,ra,dec,epsilon)
        dertau_ra, dertau_dec, dertau_tc = derivativestau(detector,ra,dec,epsilon)
        derFplus_psi, derFcross_psi = derivativepsi(detector,ra,dec,epsilon)


        for f,S in zip(fval,Sval):
            for i in range(n):
                if(i==0):
                    dfdpi = -strain(f,tau)/D
                if(i==1):
                    dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(v*Fplus-1j*Fcross)
                if(i==2):
                    dfdpi = strain(f,tau)*(5/(6*mchirp)-1j*5/4*(8*np.pi*f)**(-5/3)*mchirp**(-8/3))
                if(i==3):
                    dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_ra*(1+v**2)/2-1j*v*derFcross_ra)+strain(f,tau)*2*np.pi*1j*f*dertau_ra
                if(i==4):
                    dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_dec*(1+v**2)/2-1j*v*derFcross_dec)+strain(f,tau)*2*np.pi*1j*f*dertau_dec
                if(i==5):
                    dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
                if(i==6):
                    dfdpi = 2j*np.pi*f*(1+dertau_tc)*strain(f,tau)
                if(i==7):
                    dfdpi = -1j*strain(f,tau)

                for j in range(n):
                    if(j==0):
                        dfdpj = -strain(f,tau)/D
                    if(j==1):
                        dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(v*Fplus-1j*Fcross)
                    if(j==2):
                        dfdpj = strain(f,tau)*(5/(6*mchirp)-1j*5/4*(8*np.pi*f)**(-5/3)*mchirp**(-8/3))
                    if(j==3):
                        dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_ra*(1+v**2)/2-1j*v*derFcross_ra)+strain(f,tau)*2*np.pi*1j*f*dertau_ra
                    if(j==4):
                        dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_dec*(1+v**2)/2-1j*v*derFcross_dec)+strain(f,tau)*2*np.pi*1j*f*dertau_dec
                    if(j==5):
                        dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
                    if(j==6):
                        dfdpj = 2j*np.pi*f*(1+dertau_tc)*strain(f,tau)
                    if(j==7):
                        dfdpj = -1j*strain(f,tau)

                    F[i,j] += 4*S**(-1)*(dfdpi.real*dfdpj.real+dfdpi.imag*dfdpj.imag)*df

                    #F[i,j] += 2*S**(-1)*(dfdpi*dfdpj.conjugate()+dfdpi.conjugate()*dfdpj)*df

    I = (np.mat(F).I)
    
    relDerror.append(np.sqrt(I[0,0])/D)
    

relDerror = np.array(relDerror)
relDerror = relDerror.reshape(ramesh.shape)

print relDerror

np.savetxt('relDerror5050iota31.txt',relDerror)
    


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in sqrt


[[             nan   1.33415343e+04              nan ...,   1.63195789e+04
    1.34373857e+04              nan]
 [  1.45226927e+00   1.27986804e+00   1.32446681e+00 ...,   1.68709922e+00
    1.73552036e+00   1.45228457e+00]
 [  1.76649720e+00   1.55969363e+00   1.53336208e+00 ...,   2.05870724e+00
    2.10025886e+00   1.76650149e+00]
 ..., 
 [  2.74319135e+00   2.17807510e+00   1.84247930e+00 ...,   1.72815754e+00
    2.64701385e+00   2.74319069e+00]
 [  2.93517131e+00   2.35273874e+00   1.99769579e+00 ...,   2.25207427e+00
    3.21168015e+00   2.93517916e+00]
 [             nan   1.43890859e+04   2.56685499e+04 ...,              nan
               nan   1.45857739e+04]]


## Set iota to a fixed value 

In [72]:
#Fisher matrix for 7 unknowns: D, Mchirp, ra, dec, psi, tc, phic
fval = np.arange(1000)/1. + 20

df = (fval.max()-fval.min())/len(fval)

n = 7

# espilon for the numerical derivatives of Fplus and Fcross
epsilon = 1e-10

F = np.zeros([n,n])

for det in detectors:

    if det is 'L1':
        print 'Livingston'
        Sval = (0.8e-22*(14./(0.08+fval))**2)**2+0.67e-23**2+((fval/2000.)*3.e-23)**2
        detector = Detector('L1')
    if det is 'H1':
        print 'Hanford'
        Sval = ((1e-22*(24./(0.1+fval))**2)**2+0.7e-23**2+((fval/2000.)*3.4e-23)**2)
        detector = Detector('H1')
    if det is 'V1':
        print 'Virgo'
        Sval = (1e-22*(35./(0.1+fval))**2)**2+0.9e-23**2+((fval/2000.)*15e-23)**2
        detector = Detector('V1')
    
    tau = time_delay = detector.time_delay_from_earth_center(ra, dec, time_at_coalescence)
    
    Fplus, Fcross = detector.antenna_pattern(time_at_coalescence, ra, dec)

    #numerical derivatives
    derFplus_ra, derFcross_ra, derFplus_dec, derFcross_dec  = derivatives(detector,ra,dec,epsilon)
    dertau_ra, dertau_dec, dertau_tc = derivativestau(detector,ra,dec,epsilon)
    derFplus_psi, derFcross_psi = derivativepsi(detector,ra,dec,epsilon)


    for f,S in zip(fval,Sval):
        for i in range(n):
            if(i==0):
                dfdpi = -strain(f,tau)/D
            if(i==1):
                dfdpi = strain(f,tau)*(5/(6*mchirp)-1j*5/4*(8*np.pi*f)**(-5/3)*mchirp**(-8/3))
            if(i==2):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_ra*(1+v**2)/2-1j*v*derFcross_ra)+strain(f,tau)*2*np.pi*1j*f*dertau_ra
            if(i==3):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_dec*(1+v**2)/2-1j*v*derFcross_dec)+strain(f,tau)*2*np.pi*1j*f*dertau_dec
            if(i==4):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
            if(i==5):
                dfdpi = 2j*np.pi*f*(1+dertau_tc)*strain(f,tau)
            if(i==6):
                dfdpi = -1j*strain(f,tau)
            
            for j in range(n):
                if(j==0):
                    dfdpj = -strain(f,tau)/D
                if(j==1):
                    dfdpj = strain(f,tau)*(5/(6*mchirp)-1j*5/4*(8*np.pi*f)**(-5/3)*mchirp**(-8/3))
                if(j==2):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_ra*(1+v**2)/2-1j*v*derFcross_ra)+strain(f,tau)*2*np.pi*1j*f*dertau_ra
                if(j==3):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_dec*(1+v**2)/2-1j*v*derFcross_dec)+strain(f,tau)*2*np.pi*1j*f*dertau_dec
                if(j==4):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
                if(j==5):
                    dfdpj = 2j*np.pi*f*(1+dertau_tc)*strain(f,tau)
                if(j==6):
                    dfdpj = -1j*strain(f,tau)

                F[i,j] += 4*S**(-1)*(dfdpi.real*dfdpj.real+dfdpi.imag*dfdpj.imag)*df
                
                #F[i,j] += 2*S**(-1)*(dfdpi*dfdpj.conjugate()+dfdpi.conjugate()*dfdpj)*df

I = (np.mat(F).I)
print I

print 'The relative distance error is {0}'.format(np.sqrt(I[0,0]/D**2))
print 'The relative mchirp error is {0}'.format(np.sqrt(I[1,1]/mchirp**2))
print 'delta ra error = {0}'.format(np.sqrt(I[2,2]))
print 'delta dec error = {0}'.format(np.sqrt(I[3,3]))
print 'delta psi error = {0}'.format(np.sqrt(I[4,4]))
print "delta t_c = {0}".format(np.sqrt(I[5,5]))
print 'delta phi_c error = {0}'.format(np.sqrt(I[6,6]))

Hanford
Livingston
Virgo
[[  1.26024117e+29   2.58277451e+02  -1.94991435e+12  -3.08208131e+10
   -1.08449752e+13   1.01485919e+10  -5.27882560e+12]
 [  2.58277451e+02   1.50836889e-22   1.28440766e-14   5.25198445e-17
   -3.52145890e-14  -5.35304950e-16  -7.27812741e-13]
 [ -1.94991435e+12   1.28440766e-14   3.19500289e-04   8.25168854e-05
    1.39393889e-04  -9.09221161e-07   8.14259974e-04]
 [ -3.08208131e+10   5.25198446e-17   8.25168854e-05   5.25751498e-05
   -1.22765990e-05   6.87734058e-08   3.30658702e-04]
 [ -1.08449752e+13  -3.52145890e-14   1.39393889e-04  -1.22765991e-05
    1.23616954e-03  -8.66541401e-07   4.22375280e-04]
 [  1.01485919e+10  -5.35304950e-16  -9.09221161e-07   6.87734058e-08
   -8.66541401e-07   1.20234922e-08   4.74548324e-06]
 [ -5.27882560e+12  -7.27812741e-13   8.14259974e-04   3.30658702e-04
    4.22375280e-04   4.74548324e-06   1.05190331e-02]]
The relative distance error is 0.0862189137773
The relative mchirp error is 2.11270624412e-06
delta ra err

## The Markovic approximation with fixed iota

In [78]:
#Fisher matrix for 3 unknowns: D, v=cosi, Mchirp, ra, dec, psi, tc, phic
fval = np.arange(2020)/1. + 20

df = (fval.max()-fval.min())/len(fval)

n = 3

# espilon for the numerical derivatives of Fplus and Fcross
epsilon = 1e-10

F = np.zeros([n,n])

for det in detectors:

    if det is 'L1':
        print 'Livingston'
        Sval = (0.8e-22*(14./(0.08+fval))**2)**2+0.67e-23**2+((fval/2000.)*3.e-23)**2
        detector = Detector('L1')
    if det is 'H1':
        print 'Hanford'
        Sval = ((1e-22*(24./(0.1+fval))**2)**2+0.7e-23**2+((fval/2000.)*3.4e-23)**2)
        detector = Detector('H1')
    if det is 'V1':
        print 'Virgo'
        Sval = (1e-22*(35./(0.1+fval))**2)**2+0.9e-23**2+((fval/2000.)*15e-23)**2
        detector = Detector('V1')
    
    tau = time_delay = detector.time_delay_from_earth_center(ra, dec, time_at_coalescence)
    
    Fplus, Fcross = detector.antenna_pattern(time_at_coalescence, ra, dec)

    #numerical derivatives
    derFplus_ra, derFcross_ra, derFplus_dec, derFcross_dec  = derivatives(detector,ra,dec,epsilon)
    dertau_ra, dertau_dec, dertau_tc = derivativestau(detector,ra,dec,epsilon)
    derFplus_psi, derFcross_psi = derivativepsi(detector,ra,dec,epsilon)


    for f,S in zip(fval,Sval):
        for i in range(n):
            if(i==0):
                dfdpi = -strain(f,tau)/D
            if(i==1):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
            if(i==2):
                dfdpi = -1j*strain(f,tau)
            
            for j in range(n):
                if(j==0):
                    dfdpj = -strain(f,tau)/D
                if(j==1):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
                if(j==2):
                    dfdpj = -1j*strain(f,tau)

                F[i,j] += 4*S**(-1)*(dfdpi.real*dfdpj.real+dfdpi.imag*dfdpj.imag)*df
                
                #F[i,j] += 2*S**(-1)*(dfdpi*dfdpj.conjugate()+dfdpi.conjugate()*dfdpj)*df

I = (np.mat(F).I)
print I

print 'The relative distance error is {0}'.format(np.sqrt(I[0,0]/D**2))
print 'delta psi error = {0}'.format(np.sqrt(I[1,1]))
print 'delta phi_c error = {0}'.format(np.sqrt(I[2,2]))

Hanford
Livingston
Virgo
[[  1.08856487e+29  -9.38497209e+12  -1.09171755e+12]
 [ -9.38497209e+12   1.11490580e-03   1.29692684e-04]
 [ -1.09171755e+12   1.29692684e-04   1.77620389e-03]]
The relative distance error is 0.0801314262799
delta psi error = 0.0333902050777
delta phi_c error = 0.0421450339529
